In [7]:
from price import *
import binomial as bi
from zero import *
import numpy as np
import pandas as pd

In [12]:
# Daily Treasury Yield Curve Rate in 02/15/19
r1 = 0.0255
r2 = 0.0252
r3 = 0.0250

mp1 = round(100*np.exp(-r1*1.0), 4)
mp2 = round(100*np.exp(-r2*1.0), 4)
mp3 = round(100*np.exp(-r3*1.0), 4)
data = [
    [100, 1.00, 0, mp1],
    [100, 2.00, 0, mp2],
    [100, 3.00, 0, mp3]
]
print(data)
fr = list(map(lambda x: round(x, 4), forward_rate(data, 1.0)))
print(fr)

[[100, 1.0, 0, 97.4822], [100, 2.0, 0, 97.5115], [100, 3.0, 0, 97.531]]
[0.0255, -0.0003, -0.0002]


In [13]:
market_data = pd.DataFrame({
    "Maturity": [1.0, 2.0, 3.0],
    "Price": [mp1, mp2, mp3],
    "Yield": [r1, r2, r3],
    "ForwardRate": fr
})

print(market_data)

   Maturity    Price   Yield  ForwardRate
0       1.0  97.4822  0.0255       0.0255
1       2.0  97.5115  0.0252      -0.0003
2       3.0  97.5310  0.0250      -0.0002


In [ ]:
# calculate volatility
YCR = r1/0.0253
X = np.log(YCR)
